# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.

## Getting the data

Follow the procedure that is described below. The dataset can be found here: http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2. More specifically do the following:
- login using ssh to your cluster:  ```ssh <ssh_user_name>@<cluster_name>-ssh.azurehdinsight.net```
- download the data: ```wget http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2```
- extract the data: ```tar -jxvf confusion-2014-03-02.tbz2```
- upload the data to HDFS: ```hdfs dfs -put confusion-2014-03-02/confusion-2014-03-02.json /tmp/```


After you have uploaded the dataset into the Azure Blob, upload this notebook onto the Spark Jupyter server  (`https://<cluster-name>.azurehdinsight.net/jupyter`).

## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in this link: http://lars.yencken.org/datasets/languagegame/


## Preprocessing commands
In your newly created notebook run these commands in order to have the dataset into an RDD:

In [1]:
import json

path = "/tmp/confusion-2014-03-02.json"
raw_data = sc.textFile(path)
dataset = raw_data.map(json.loads).cache()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1540739820917_0007,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


After that you will be able to run the queries of the moodle question of this week. The RDD that you have to perform your queries on is the ```dataset``` one. For example, the following command returns one element of the dataset:

In [2]:
dataset.take(1)

[{'choices': ['Maori', 'Mandarin', 'Norwegian', 'Tongan'], 'guess': 'Norwegian', 'target': 'Norwegian', 'date': '2013-08-19', 'country': 'AU', 'sample': '48f9c924e0d98c959d8a6f1862b3ce9a'}]

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query as well as the productivity time. That means the development time of each query (time elapsed before you start writing the query, and the time at which the correct, final query is ready). Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

In [3]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise():
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))

If you want to have a part of the dataset in order to run faster experiments then run the cell below and run your queries against the ```dataset_part``` RDD. However your final answer, the format of your final query and the productivity time should be run/measured against the whole dataset.

In [4]:
dataset_part = dataset.sample(False, 0.0001).cache()

## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Russian.

In [5]:
start_exercise()

In [6]:
dataset \
    .filter(lambda o: o['guess'] == o['target']) \
    .filter(lambda o: o['target'] == 'Russian') \
    .count()

290818

In [7]:
finish_exercise()

This exercise took 47s

## Assignment 2
Return the number of distinct "target" languages.

In [8]:
start_exercise()

In [9]:
dataset \
    .map(lambda o: o['target']) \
    .distinct() \
    .count()

78

In [10]:
finish_exercise()

This exercise took 15s

## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the top three games where the guessed language is correct (equal to the target one) ordered by language (ascending), then by country (ascending), then by date (ascending).

In [11]:
start_exercise()

In [12]:
dataset \
    .filter(lambda o: o['guess'] == o['target']) \
    .map(lambda o: ((o['target'], o['country'], o['date']), o)) \
    .sortByKey() \
    .map(lambda t: t[1]) \
    .take(3)

[{'choices': ['Albanian', 'Macedonian'], 'guess': 'Albanian', 'target': 'Albanian', 'date': '2013-09-04', 'country': 'A1', 'sample': '00b85faa8b878a14f8781be334deb137'}, {'choices': ['Albanian', 'Bulgarian', 'Indonesian', 'Portuguese'], 'guess': 'Albanian', 'target': 'Albanian', 'date': '2013-09-05', 'country': 'A1', 'sample': 'efcd813daec1c836d9f030b30caa07ce'}, {'choices': ['Albanian', 'Hindi', 'Swahili'], 'guess': 'Albanian', 'target': 'Albanian', 'date': '2013-09-08', 'country': 'A1', 'sample': '13722ceed1eede7ba597ade9b4cb9807'}]

In [13]:
finish_exercise()

This exercise took 95s

## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [14]:
start_exercise()

In [15]:
dataset \
    .map(lambda o: ((o['country'], o['target']), 1)) \
    .reduceByKey(lambda i1, i2: i1 + i2) \
    .map(lambda t: (t[1], t[0])) \
    .sortByKey(ascending=False) \
    .take(3)

[(112934, ('US', 'French')), (112007, ('US', 'German')), (110919, ('US', 'Spanish'))]

In [16]:
finish_exercise()

This exercise took 15s

## Assignment 5
Find the overall percentage of correct guesses when the first answer (amongst the array of possible answers) was the correct one.

In [17]:
start_exercise()

In [18]:
n = dataset \
    .filter(lambda o: o['guess'] == o['target']) \
    .filter(lambda o: o['target'] == o['choices'][0]) \
    .count()
n_tot = dataset.count()
n / n_tot

0.25603983084476356

In [19]:
finish_exercise()

This exercise took 23s

## Assignment 6
Sort the languages by decreasing overall percentage of correct guesses and return the first three languages.

In [20]:
start_exercise()

In [21]:
dataset \
    .map(lambda o: (o['target'], (1, 1 if o['target'] == o['guess'] else 0))) \
    .reduceByKey(lambda t1, t2: (t1[0] + t2[0], t1[1] + t2[1])) \
    .map(lambda t: (t[1][1]/t[1][0], t[0])) \
    .sortByKey(ascending=False) \
    .take(3)

[(0.9382414927447232, 'French'), (0.9197634593055483, 'German'), (0.8956432115670598, 'Spanish')]

In [22]:
finish_exercise()

This exercise took 15s

## Assignment 7
Return the number games played on the latest day.

In [23]:
start_exercise()

In [24]:
latest_day = dataset \
    .map(lambda o: (o['date'], 0)) \
    .sortByKey(False) \
    .take(1)[0][0]
print("Latest day:", latest_day)
dataset \
    .filter(lambda o: o['date'] == latest_day) \
    .count()

Latest day: 2014-03-01
65653

In [25]:
finish_exercise()

This exercise took 55s